In [ ]:
11.17第二节 图与会话
2-小练习然后描述图的形式，并简要画出从1到3的累加的图的形式。
import tensorflow as tf
a=0
for i in range(1,4):
​    a=tf.add(a,i)
with tf.Session() as sess:
​    print(sess.run(a))
小练习**：

尝试为不同设备分配不同计算节点，并打印出设备信息。
应该不用手动打印
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    print(sess.run(c))
在一个notebook文件中构建一张图，实现两个数的加法操作，并在两个不同的会话中执行图。
#create a graph

a=tf.add(10,20)

#create session1

sess1=tf.Session()

#execute session1

res1=sess1.run(a)

#print the result of session1

print(res1)

 #create session2 using context manager

 with tf.Session() as sess2:

​            print(sess2.run(a))

In [44]:
#第三节图的边与节点
#小练习设计一个函数，要求实现：可以根据输入张量
#输出shape完成一样的元素为全1的张量。
#提示，使用`tf.ones`函数可根据形状生成全1张量。
def tran(a):
    a=tf.constant([[1, 2, 3], [1, 2, 3]])
    with tf.Session() as sess:
        print(sess.run(tf.ones_like(a)))
        print(sess.run(tf.zeros_like(a)))
a=tf.constant([[1, 2, 3], [1, 2, 3]])
tran(a)

[[1 1 1]
 [1 1 1]]
[[0 0 0]
 [0 0 0]]


In [67]:
#z=tf.constant([1, 2, 3], dtype=tf.string)
#with tf.Session() as sess:
 #   print(sess.run(z))
    #这个例子是错的


In [113]:
#new_t = tf.reshape(new_t, [-1]) 
#tf.reshape(new_t, [-1])将张量变为一维张量
import tensorflow as tf
t1 = [[1, 2, 3], [4, 5, 6]]
t2 = [[7, 8, 9], [10, 11, 12]]
print(tf.concat([t1,t2],0))
#如果concat_dim是0，那么在某一个shape
#的第一个维度上连，对应到实际，就是叠放到列上
with tf.Session() as sess:
    print(sess.run(tf.concat([t1,t2],0)))
    # 如果concat_dim是1，
    #那么在某一个shape的第二个维度上连
    print(sess.run(tf.concat([t1,t2],1)))

Tensor("concat_15:0", shape=(4, 3), dtype=int32)
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]
[[ 1  2  3  7  8  9]
 [ 4  5  6 10 11 12]]


In [115]:
import tensorflow as tf
a=[[[1,1,1],[2,2,2]],
    [[3,3,3],[4,4,4]],
    [[5,5,5],[6,6,6]]]
with tf.Session() as sess:
    print(sess.run(tf.slice(a,[1,0,0],[1,1,3])))

[[[3 3 3]]]


In [4]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split

#任务一：将鸢尾花数据集安装8 : 2的比例划分成训练集与验证集
iris=datasets.load_iris()
x=iris.data#获取花卉两列数据集(二维矩阵150，4)
y=iris.target#一维向量,鸢尾花的类别
x_train,x_validation,y_train,y_validation=train_test_split(x,y,test_size=0.2,random_state=42)
#x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2,random_state=42)
#任务二：设计模型，训练模型并验证模型

from collections import Counter
def counter(arr):
    return Counter(arr).most_common(1) # 返回出现频率最高的一个数
accuracy=0
Ntest=len(x_validation)

for i in range(Ntest):
    distance=tf.reduce_sum(tf.abs(tf.add(x_train,tf.negative(x_validation[i]))),reduction_indices=1)
    #print(distance)
    value_ = tf.nn.top_k(-distance, k=1).values
    indices_ = tf.nn.top_k(-distance, k=1).indices#前k个近邻索引
    with tf.Session() as sess:
        sess.run(distance)
        sess.run(-value_)
        indices=np.array(sess.run(indices_))
        #获取前k个近邻的标签
    a=[]
    for j in range(len(indices)):
        a.append(y_train[indices[j]])
    #print(a)
    z=counter(a)#得到a中出现次数最多的一个元素，及出现次数
    pred_class_label=z[0][0]#max(set(a))
    true_class_label = y_validation[i]
  
    #print("Test", i, "Predicted Class Label:", pred_class_label,
        #"True Class Label:",  true_class_label )
    if pred_class_label == true_class_label:
        accuracy += 1
print((accuracy/30)*100,'%')


100.0 %


In [8]:
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split

#任务一：将鸢尾花数据集安装8 : 2的比例划分成训练集与验证集
iris=datasets.load_iris()
x=iris.data#获取花卉两列数据集(二维矩阵150，4)
y=iris.target#一维向量,鸢尾花的类别
x_train,x_validation,y_train,y_validation=train_test_split(x,y,test_size=0.2,random_state=42)
#x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2,random_state=42)

#任务二，三：设计模型，训练模型
from collections import Counter
def counter(arr):
    return Counter(arr).most_common(1) # 返回出现频率最高的一个数

Ntest=len(x_validation)
def k_nn(k_num,Ntest):
    accuracy=0
    for i in range(Ntest):
        #计算l1距离
        distance=tf.reduce_sum(tf.abs(tf.add(x_train,tf.negative(x_validation[i]))),reduction_indices=1)
        #print(distance)
        
        value_ = tf.nn.top_k(-distance, k=k_num).values
        indices_ = tf.nn.top_k(-distance, k=k_num).indices#前k个近邻索引
        
        with tf.Session() as sess:
            sess.run(distance)
            sess.run(-value_)
            indices=np.array(sess.run(indices_))
        
        #获取前k个近邻的标签
        a=[]
        for j in range(len(indices)):
            a.append(y_train[indices[j]])
            
        #得到a中出现次数最多的一个元素，及出现次数
        z=counter(a)
        
        print('indices',indices,'a',a,'z',z)
        pred_class_label=z[0][0]#max(set(a))
        true_class_label = y_validation[i]
  
        #print("Test", i, "Predicted Class Label:", pred_class_label,
        #"True Class Label:",  true_class_label )
        if pred_class_label == true_class_label:
            accuracy += 1
    print('k',k_num,(accuracy/30)*100,'%')
#验证模型
for m in range(1,5):
    k_nn(m,Ntest)

indices [79] a [1]
indices [48] a [0]
indices [24] a [2]
indices [90] a [1]
indices [12] a [1]
indices [114] a [0]
indices [95] a [1]
indices [85] a [2]
indices [110] a [1]
indices [118] a [1]
indices [87] a [2]
indices [104] a [0]
indices [7] a [0]
indices [55] a [0]
indices [98] a [0]
indices [6] a [1]
indices [76] a [2]
indices [53] a [1]
indices [20] a [1]
indices [107] a [2]
indices [3] a [0]
indices [42] a [2]
indices [58] a [0]
indices [107] a [2]
indices [37] a [2]
indices [87] a [2]
indices [109] a [2]
indices [61] a [2]
indices [104] a [0]
indices [55] a [0]
k 1 100.0 %
indices [79 90] a [1, 1]
indices [48 14] a [0, 0]
indices [24 21] a [2, 2]
indices [90 79] a [1, 1]
indices [12 80] a [1, 1]
indices [114   7] a [0, 0]
indices [ 95 108] a [1, 1]
indices [85 61] a [2, 2]
indices [110  68] a [1, 2]
indices [118  25] a [1, 1]
indices [87 83] a [2, 2]
indices [104  55] a [0, 0]
indices [ 7 14] a [0, 0]
indices [ 55 104] a [0, 0]
indices [98 84] a [0, 0]
indices [ 6 11] a [1, 1]
i